In [1]:
from dataclasses import dataclass
from re import template
from typing import List, Dict, Optional, Any, Union
from pydantic import BaseModel, Field
from typing import ClassVar
import os
from pydantic import BaseModel, Field
from typing import ClassVar
import re

from enum import Enum


class Flag(Enum):
    NONE = "none"
    RED = "red"
    ORANGE = "orange"
    GREEN = "green"
    BLUE = "blue"
    PINK = "pink"
    TURQUOISE = "turquoise"
    PURPLE = "purple"

    @staticmethod
    def from_str(s: str):
        s = s.lower()
        for flag in Flag:
            if flag.value == s:
                return flag
        raise ValueError(f"{s} is not a valid flag.")


class CardState(Enum):
    NEW = "new"
    LEARNING = "learning"
    REVIEW = "review"
    SUSPENDED = "suspended"
    BURIED = "buried"

    @staticmethod
    def from_str(s: str):
        s = s.lower()
        for state in CardState:
            if state.value == s:
                return state
        raise ValueError(f"{s} is not a valid state.")


@dataclass(frozen=False)
class Card:
    """
    A Card is a representation of a flashcard, containing a question and an answer. The card is uniquely identified by the id.

    Properties:
      id (str): The id uniquely identifies the card. It is represented as "card_xxxx_xxxx", with x being hexadecimal digits.
          The id is the only way to identify a card.
      question (str): The question (frontside) of the card.
      answer (str): The answer (frontside) of the card.
      flag (str): The flag of the card. **Must** be one of:
          none, red, orange, green, blue, pink, turquoise, purple
      cardState (str): The state of the card in the flashcard system. **Must** be one of:
          new, learning, review, suspended, buried
    """
    id: int
    deck: "Deck"
    question: str
    answer: str
    flag: Flag
    cardState: CardState

    def __str__(self):
        hex_str = f"{self.id:08x}"  # pad to 8 hex digits
        hex_id = f"card_{hex_str[:4]}_{hex_str[4:]}"
        s = f"""Card {hex_id} from the deck {self.deck}
Question:
{self.question}

Answer:
{self.answer}

Flag: {self.flag.value}
Card State: {self.cardState.value}"""
        return s


@dataclass(frozen=False)
class Deck:
    """
    A Deck represents a collection of flashcards.

    Properties:
       id (str): The id uniquely identifies the deck. It is represented as "deck_xxxx_xxxx", with x being hexadecimal digits.
          The id is the only way to identify a card. It is assigned randomly, there is no way to guess it!
       name (str): The name of the deck. This is **not** the id, and is **not** sufficient to address decks.
       cards (List[Card]): The cards contained in the deck. The order has no meaning.
    """
    id: int
    name: str
    cards: List[Card]

    def __str__(self):
        hex_str = f"{self.id:08x}"  # pad to 8 hex digits
        hex_id = f"deck_{hex_str[:4]}_{hex_str[4:]}"
        s = f"""Deck '{self.name}' (id: {hex_id}) containing {len(self.cards)} cards"""
        return s

    def copy(self, new_flashcard_manager: "FlashcardManager"):
        new_deck = new_flashcard_manager.add_deck(self.name)
        for card in self.cards:
            new_flashcard_manager.add_card(new_deck, card.question, card.answer, card.flag, card.cardState)


class FlashcardManager:
    __decks_by_id: dict[int, Deck]
    __cards_by_id: dict[int, Card]
    __decks_by_name: dict[str, Deck]

    __CARD_ID_REGEX: ClassVar[re.Pattern] = re.compile(r"^card_[0-9a-fA-F]{4}_[0-9a-fA-F]{4}$")
    __DECK_ID_REGEX: ClassVar[re.Pattern] = re.compile(r"^deck_[0-9a-fA-F]{4}_[0-9a-fA-F]{4}$")

    def __init__(self):
        self.__decks_by_id = {}
        self.__cards_by_id = {}
        self.__decks_by_name = {}

    def get_deck_by_name(self, deck_name: str) -> Deck:
        return self.__decks_by_name[deck_name]

    def get_decks(self) -> list[Deck]:
        return list(self.__decks_by_id.values())

    def add_deck(self, deck_name: str):
        if deck_name in self.__decks_by_name:
            raise ValueError(f"Deck '{deck_name}' already exists.")

        deck = Deck(name=deck_name, id=self.create_deck_id(), cards=[])
        self.__decks_by_id[deck.id] = deck
        self.__decks_by_name[deck.name] = deck

        return deck

    def delete_deck(self, deck):
        if deck not in self.__decks_by_id.values():
            return

        self.__decks_by_name.pop(deck.name)
        self.__decks_by_id.pop(deck.id)

        for card in deck.cards:
            self.__cards_by_id.pop(card.id)

    def add_card(self, deck: Deck, question: str, answer: str, flag: Union[Flag, str],
                 card_state: Union[CardState, str]):
        if isinstance(card_state, str):
            card_state = CardState.from_str(card_state)
        if isinstance(flag, str):
            flag = Flag.from_str(flag)

        card = Card(id=self.create_card_id(), question=question, answer=answer, flag=flag, cardState=card_state,
                    deck=deck)
        self.__cards_by_id[card.id] = card
        deck.cards.append(card)
        return card

    def delete_card(self, deck: Deck, card: Card):
        deck.cards.remove(card)  # throws error if not present
        self.__cards_by_id.pop(card.id)

    def create_deck_id(self) -> int:
        attempt = 0
        while True:
            attempt += 1
            random_bytes = os.urandom(4)
            random_int = int.from_bytes(random_bytes, byteorder="big")
            if random_int not in self.__decks_by_id:
                return random_int
            if attempt >= 100:
                raise RuntimeError(f"{attempt} attempts of generating a new, unique card id failed.")

    def create_card_id(self) -> int:
        attempt = 0
        while True:
            attempt += 1
            random_bytes = os.urandom(4)
            random_int = int.from_bytes(random_bytes, byteorder="big")
            if random_int not in self.__cards_by_id:
                return random_int
            if attempt >= 100:
                raise RuntimeError(f"{attempt} attempts of generating a new, unique card id failed.")

    def get_deck_by_id(self, deck_id: int) -> "Deck":
        res = self.__decks_by_id[deck_id]
        if res is None:
            raise RuntimeError(f"{deck_id} not found.")
        return res

    def get_deck_by_id_string(self, deck_id_string: str) -> "Deck":
        if not isinstance(deck_id_string, str):
            raise ValueError("Deck id must be a string in the format 'deck_xxxx_xxxx'.")
        if not self.__DECK_ID_REGEX.fullmatch(deck_id_string):
            raise ValueError(
                f"Invalid deck id format: '{deck_id_string}'. Expected format: 'deck_xxxx_xxxx' with 8 hex digits.")

        hex_part = deck_id_string[5:].replace("_", "")
        return self.get_deck_by_id(int(hex_part, 16))

    def get_card_by_id(self, card_id: int) -> "Card":
        res = self.__cards_by_id[card_id]
        if res is None:
            raise RuntimeError(f"{card_id} not found.")
        return res

    def get_card_by_id_string(self, card_id_string: str) -> "Card":
        if not isinstance(card_id_string, str):
            raise ValueError("Card id must be a string in the format 'card_xxxx_xxxx'.")
        if not self.__CARD_ID_REGEX.fullmatch(card_id_string):
            raise ValueError(
                f"Invalid card id format: '{card_id_string}'. Expected format: 'card_xxxx_xxxx' with 8 hex digits.")

        hex_part = card_id_string[5:].replace("_", "")
        return self.get_card_by_id(int(hex_part, 16))

    def copy(self):
        new_manager = FlashcardManager()
        for deck in self.get_decks():
            deck.copy(new_manager)
        return new_manager

    def __str__(self):
        if len(self.get_decks()) == 0:
            return "Empty Flashcard Manager."
        return f"Flashcard Manager with the following decks:\n{'\n'.join(['* ' + str(deck) for deck in self.get_decks()])}\n"




In [2]:
# generated by datamodel-codegen:
#   filename:  test_schema.json
#   timestamp: 2025-05-19T10:57:24+00:00

# from __future__ import annotations

from typing import Any, Dict, List, Optional, Union

from pydantic import BaseModel, Extra, Field, constr


class Test_Card(BaseModel):
    class Config:
        extra = "forbid"

    question: str
    answer: str
    flag: str
    cardState: str


class Test_TestDecks(BaseModel):
    class Config:
        extra = "forbid"

    name: str
    cards: List[Test_Card]


class Test_DummyEnvironments(BaseModel):
    class Config:
        extra = "forbid"

    decks: List[str]


class Test_Card_Fuzzy(BaseModel):
    class Config:
        extra = "forbid"

    question: str
    answer: str
    flag: str
    cardState: str
    field__fuzzymatch: List[str] = Field([], alias='__fuzzymatch')


class Test_Deck(BaseModel):
    class Config:
        extra = "forbid"

    name: str
    cards: List[Test_Card_Fuzzy]


class Test_ExpectedResult(BaseModel):
    decks: List[Union[str, Test_Deck]]


class Test_Test(BaseModel):
    class Config:
        extra = "forbid"

    name: str
    description: Optional[str] = None
    environment: str
    queries: List[List[str]]
    params: Optional[Dict[str, Any]] = None
    expected_result: Test_ExpectedResult


class Test_QuestionAnsweringItem(BaseModel):
    class Config:
        extra = "forbid"

    name: str
    description: str
    environment: str
    queries: List[List[str]]
    expected_answer: str


class Test_Data(BaseModel):
    test_decks: Optional[Dict[str, Test_TestDecks]] = None
    dummy_environments: Optional[Dict[str, Test_DummyEnvironments]] = None
    tests: Optional[List[Test_Test]] = None
    question_answering: Optional[Dict[str, List[Test_QuestionAnsweringItem]]] = None


In [3]:
import pathlib


def load_data(path: str):
    json_path = pathlib.Path(path)
    data = Test_Data.model_validate_json(json_path.read_text(encoding="utf-8"))

    return data


test_data_path = "../tests/data/tests.json"
test_data = load_data(test_data_path)

In [4]:
# Create a FlashcardManager from the test inputs, containing the test decks.
fcm = FlashcardManager()

for test_deck in test_data.test_decks.values():
    deck = fcm.add_deck(test_deck.name)
    for card in test_deck.cards:
        fcm.add_card(deck, card.question, card.answer, card.flag, card.cardState)

del test_deck, deck, card  # man i hate python scopes.

In [5]:
[it.name for it in fcm.get_decks()]

['Empty Deck',
 'Latin Literature',
 'Java Programming',
 'Java Programming 10',
 'python']

In [6]:
fcm.get_deck_by_name("Latin Literature")

Deck(id=3645446090, name='Latin Literature', cards=[Card(id=1521863802, deck=..., question='Who was Virgil?', answer="A Roman poet known for the 'Aeneid'", flag=<Flag.NONE: 'none'>, cardState=<CardState.NEW: 'new'>), Card(id=3800122454, deck=..., question="What is the 'Aeneid'?", answer="An epic poem about Aeneas's journey", flag=<Flag.NONE: 'none'>, cardState=<CardState.NEW: 'new'>), Card(id=453338490, deck=..., question='Who was Horace?', answer='A leading Roman lyric poet', flag=<Flag.NONE: 'none'>, cardState=<CardState.NEW: 'new'>), Card(id=1252490680, deck=..., question="What type of work is 'Satires' by Horace?", answer='A collection of satirical poems', flag=<Flag.NONE: 'none'>, cardState=<CardState.NEW: 'new'>), Card(id=2291915975, deck=..., question='Who was Ovid?', answer="A Roman poet famous for 'Metamorphoses'", flag=<Flag.NONE: 'none'>, cardState=<CardState.NEW: 'new'>), Card(id=3104472892, deck=..., question="What is 'Metamorphoses'?", answer='A narrative poem of mytholog

In [7]:
print(fcm.get_deck_by_name("Latin Literature").cards[3])

Card card_4aa7_7db8 from the deck Deck 'Latin Literature' (id: deck_d949_17ca) containing 30 cards
Question:
What type of work is 'Satires' by Horace?

Answer:
A collection of satirical poems

Flag: none
Card State: new


In [8]:
import inspect

# function_registry.py
llm_commands = {}

In [9]:


def llm_command(func):
    llm_commands[func.__name__] = func
    return func


def get_llm_commands():
    res = []
    for cmnd_name, llm_command in llm_commands.items():
        params = []
        sig = inspect.signature(llm_command)
        for name, param in sig.parameters.items():
            if name == "self":
                continue
            params += [f"{name}: {param.annotation.__name__}"]

        if sig.return_annotation is None:
            returnType = "None"
        else:
            returnType = sig.return_annotation

        signature = f"{cmnd_name}({", ".join(params)}) -> {getattr(returnType, '__name__', returnType)}"
        signature = signature.replace("_empty", "<unspecified>")
        docs = llm_command.__doc__.strip("\n")
        res += [f"{signature}\n{docs}"]

    s = "\n\n".join(res)
    s = s.replace("__main__.", "") # remove unnecessary main references
    return s


In [10]:
from typing import TypeVar, Generic, List, Iterator

T = TypeVar("T")

CARD_STREAM_CHUNK_SIZE = 5


class ChunkedCards(Generic[T]):
    def __init__(self, items: List[T], chunk_size: int = CARD_STREAM_CHUNK_SIZE):
        self.items = items
        self.chunk_size = chunk_size

    def __iter__(self) -> Iterator[List[T]]:
        for i in range(0, len(self.items), self.chunk_size):
            yield self.items[i:i + self.chunk_size]

    def __len__(self) -> int:
        return (len(self.items) + self.chunk_size - 1) // self.chunk_size


class LLMInteractor:
    flashcard_manager: FlashcardManager

    def __init__(self, flashcard_manager: FlashcardManager):
        self.flashcard_manager = flashcard_manager

    # currently, there is no "Decks" or "FlashcardProvider" class, instead decks and cards are managed by the
    # static method of deck and card -> terrible idea. Ooops.
    @llm_command
    def list_decks(self, ) -> list[Deck]:
        """
        List all available decks. Necessary to get the ids of the decks.
        """
        return self.flashcard_manager.get_decks()

    @llm_command
    def create_deck(self, name: str) -> Deck:
        """
        Create a new deck with the given name. The name must be a non-empty string.
        There may be no deck with the same name.
        The Deck containing the generated id is returned.
        """
        if not isinstance(name, str) or not name.strip():
            raise ValueError("Deck name must be a non-empty string.")

        deck = self.flashcard_manager.add_deck(name)
        return deck

    @llm_command
    def delete_deck(self, deck_id_str: str) -> None:
        """
        Delete a deck by its id. The id must be a string in the format 'deck_xxxx_xxxx'.
        """
        deck = self.flashcard_manager.get_deck_by_id_string(deck_id_str)
        self.flashcard_manager.delete_deck(deck)

    @llm_command
    def list_cards(self, deck_id_str: str, search_substring: str = None) -> ChunkedCards[Card]:
        """
        List all cards in a deck, optionally filtering by a substring in the question.
        The id must be a string in the format 'deck_xxxx_xxxx'.
        The cards will be returned in batches of 5.
        """
        deck = self.flashcard_manager.get_deck_by_id_string(deck_id_str)
        cards = deck.cards
        if search_substring:
            if not isinstance(search_substring, str):
                raise ValueError("Search substring must be a string.")
            cards = [c for c in cards if search_substring.lower() in c.question.lower()]
        return ChunkedCards(cards)

    @llm_command
    def add_card(self, deck_id_str: str, question: str, answer: str, state: str, flag: str) -> None:
        """
        Create a new card in a deck. The deck id must be a string in the format 'deck_xxxx_xxxx'.
        The question, answer, state, and flag must all be non-empty strings.
        The state must be a valid CARD_STATE.
        The flag must be a valid CARD_FLAG.
        """
        if not all(isinstance(x, str) and x.strip() for x in [question, answer, state, flag]):
            raise ValueError("Question, answer, state, and flag must all be non-empty strings.")
        deck = self.flashcard_manager.get_deck_by_id_string(deck_id_str)
        self.flashcard_manager.add_card(deck, question, answer, flag, state)

    @llm_command
    def edit_card_question(self, card_id_str: str, new_question: str) -> None:
        """
        Edit the question of a card. The id must be a string in the format 'card_xxxx_xxxx'.
        """
        if not isinstance(new_question, str) or not new_question.strip():
            raise ValueError("New question must be a non-empty string.")
        card = self.flashcard_manager.get_card_by_id_string(card_id_str)
        card.question = new_question

    @llm_command
    def edit_card_answer(self, card_id_str: str, new_answer: str) -> None:
        """
        Edit the answer of a card. The id must be a string in the format 'card_xxxx_xxxx'.
        """
        if not isinstance(new_answer, str) or not new_answer.strip():
            raise ValueError("New answer must be a non-empty string.")
        card = self.flashcard_manager.get_card_by_id_string(card_id_str)
        card.answer = new_answer

    @llm_command
    def edit_card_flag(self, card_id_str: str, new_flag: str) -> None:
        """
        Edit the flag of a card. The id must be a string in the format 'card_xxxx_xxxx'.
        """
        if not isinstance(new_flag, str) or not new_flag.strip():
            raise ValueError("New flag must be a non-empty string.")
        card = self.flashcard_manager.get_card_by_id_string(card_id_str)
        card.flag = new_flag

    @llm_command
    def edit_card_state(self, card_id_str: str, new_state: str) -> None:
        """
        Edit the state of a card. The id must be a string in the format 'card_xxxx_xxxx'.
        """
        if not isinstance(new_state, str) or not new_state.strip():
            raise ValueError("New state must be a non-empty string.")
        card = self.flashcard_manager.get_card_by_id_string(card_id_str)
        card.cardState = new_state

    @llm_command
    def delete_card(self, card_id_str: str) -> None:
        """
        Delete a card by its id. The id must be a string in the format 'card_xxxx_xxxx'.
        """
        card = self.flashcard_manager.get_card_by_id_string(card_id_str)
        if not card.deck:
            raise RuntimeError(f"Card '{card_id_str}' does not have an associated deck.")
        deck = card.deck
        if card not in deck.cards:
            raise RuntimeError(f"Card '{card_id_str}' is not present in its deck.")
        self.flashcard_manager.delete_card(deck, card)

    # _virtual_decks: dict[str, list] = {}
    # _active_virtual_deck: str = None
    #
    #
    # #@llm_command TODO later
    # def create_virtual_deck(self,virtual_deck_name: str):
    #     if not isinstance(virtual_deck_name, str) or not virtual_deck_name.strip():
    #         raise ValueError("Virtual deck name must be a non-empty string.")
    #     if virtual_deck_name in _virtual_decks:
    #         raise ValueError(f"Virtual deck '{virtual_deck_name}' already exists.")
    #     _virtual_decks[virtual_deck_name] = []
    #
    #
    # #@llm_command TODO later
    # def activate_virtual_deck(self,virtual_deck_name: str):
    #     global _active_virtual_deck
    #     if not isinstance(virtual_deck_name, str) or not virtual_deck_name.strip():
    #         raise ValueError("Virtual deck name must be a non-empty string.")
    #     if virtual_deck_name not in _virtual_decks:
    #         raise KeyError(f"Virtual deck '{virtual_deck_name}' does not exist.")
    #     _active_virtual_deck = virtual_deck_name
    #

    # #@llm_command TODO later
    # def virtual_deck_add_card(self,card_id_str: str):
    #     if _active_virtual_deck is None:
    #         raise RuntimeError("No active virtual deck. Please activate a virtual deck first.")
    #     card = Card.get_card_by_id_string(card_id_str)
    #     if card in _virtual_decks[_active_virtual_deck]:
    #         raise ValueError(f"Card '{card_id_str}' is already in the active virtual deck.")
    #     _virtual_decks[_active_virtual_deck].append(card)
    #
    #
    # #@llm_command TODO later
    # def virtual_deck_remove_card(self,card_id_str: str):
    #     if _active_virtual_deck is None:
    #         raise RuntimeError("No active virtual deck. Please activate a virtual deck first.")
    #     card = Card.get_card_by_id_string(card_id_str)
    #     if card not in _virtual_decks[_active_virtual_deck]:
    #         raise ValueError(f"Card '{card_id_str}' is not in the active virtual deck.")
    #     _virtual_decks[_active_virtual_deck].remove(card)
    #
    #
    # #@llm_command TODO later
    # def virtual_deck_list(self,):
    #     if _active_virtual_deck is None:
    #         raise RuntimeError("No active virtual deck. Please activate a virtual deck first.")
    #     return ChunkedCards(_virtual_decks[_active_virtual_deck])
    #
    #
    # #@llm_command TODO later
    # def deactivate_virtual_deck(self,):
    #     global _active_virtual_deck
    #     if _active_virtual_deck is None:
    #         raise RuntimeError("No virtual deck is currently active.")
    #     _active_virtual_deck = None

In [11]:
import re
import ast
import openai


class LLMCommunicator:
    messages: list[dict[str, str]]
    model: str
    temperature: float
    max_tokens: Optional[int]

    def __init__(self, model: str, temperature: float, max_tokens: Optional[int] = None):
        self.model = model
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.client = openai.OpenAI(
            api_key="lm-studio",
            base_url="http://localhost:1234/v1"
        )
        self.messages = []

    def send_message(self, message: str) -> str:
        request_message = {"role": "user", "content": message}
        self.messages.append(request_message)

        response = self.client.chat.completions.create(
            model=self.model,
            messages=self.messages,
            temperature=self.temperature,
            max_tokens=self.max_tokens,
        )
        msg = response.choices[0].message
        self.messages.append({"role": msg.role, "content": msg.content})
        return msg.content


class TaskExecutor:
    __first_message: str = None
    function_map = llm_commands

    @staticmethod
    def get_first_message(user_prompt: str):
        if TaskExecutor.__first_message is not None:
            return TaskExecutor.__first_message + user_prompt.strip()

        available_functions = get_llm_commands()

        template = f"""You are an assistant for a flashcard learning system. The system manages Cards, that are collected in Decks.

About Cards:
{Card.__doc__.strip("\n")}

About Decks:
{Deck.__doc__.strip("\n")}

You can interact with the system by calling specific Python functions, each of which performs an action. The available actions are:
{available_functions}

_________________________________

First, you **have to** think about what the user wants, which information you need, and make a rough plan of your actions. Almost always you will need further information (e.g., deck ids, card ids, or card content). In this case, you will request the information.
After reasoning, output the steps you want to execute **now** in the following format:

<execute>
* function_call_1(arguments)
* function_call_2(arguments)
...
</execute>

The system will then execute your commands, and return an python array of results:
["result_of_call_1", "result_of_call_2", ...]

If no further actions are needed, please return an empty execute block:

<execute>
</execute>

For example, if the user prompt was:
"Create a new deck with the name Astrology and add What is the largest planet in our solar system? and Jupiter to it. Flag it as Purple."
Your steps should be:
* Create a new deck with the name Astrology. Wait for the output to get the id of this new deck.
* Add a card with the given question, answer and flag to the deck.

So the first execution plan would be:
<execute>
* create_deck("Astrology")
</execute>

The system then would answer you the information of the newly created deck, e.g.:
["Deck 'Astrology' (id: deck_9874_2787)"]

Then, the next execution plan would be:
<execute>
* add_card(deck_id_str="deck_9874_2787", question="What is the largest planet in our solar system?", answer="Jupiter", state="new", flag="purple")
</execute>

The system then provides you with an empty response.
Then, you have achieved your task, and return:

<execute>
</execute>


The user prompt is:
"""

        TaskExecutor.__first_message = template
        return TaskExecutor.__first_message + user_prompt.strip()

    @staticmethod
    def execute_prompt(flashcard_manager: FlashcardManager, user_prompt: str):
        llm_interactor = LLMInteractor(flashcard_manager)
        llm_communicator = LLMCommunicator("qwen3-8b", 0.8)
        # llm_communicator = LLMCommunicator("qwen2.5-14b-instruct", 0.8)
        error_count = 0

        message_to_send = TaskExecutor.get_first_message(user_prompt)

        while True:
            try:
                print("\n=========== REQUEST ===========:")
                print(message_to_send)
                answer = llm_communicator.send_message(message_to_send)
                print("\n=========== RESPONSE ==========")
                print(answer)
                evaluated = TaskExecutor.evaluate_llm_response(llm_interactor, answer)
                if len(evaluated) == 0:
                    return
                else:
                    message_to_send = str([str(it) for it in evaluated])
            except Exception as e:
                print("\nException raised:")
                print(e)
                error_count += 1
                if error_count > 10:
                    raise RuntimeError("Too many errors. Abort execution.")
                message_to_send = f"""An error occured: {e} Please try again!"""

    @staticmethod
    def evaluate_llm_response(llm_interactor: LLMInteractor, response: str):
        # Extract the execution plan block
        match = re.search(r"<execute>(.*?)</execute>", response, re.DOTALL)
        if not match:
            raise ValueError("No execute block found in response. Remember to use <execute>...</execute> to mark your execution plan, and send an empty block to indicate that you do not wish to take any further action.")
        plan = match.group(1)

        results = []
        for line in plan.splitlines():
            line = line.strip()
            if not line.strip():
                continue
            command = line[1:].strip()
            # Parse function name and arguments
            m = re.match(r"(\w+)\((.*)\)", command)
            if not m:
                raise ValueError(f"Invalid command syntax: {command}")
            func_name, arg_str = m.groups()
            if func_name not in TaskExecutor.function_map:
                raise ValueError(f"Unknown function: {func_name}")
            # Parse arguments safely
            args = []
            kwargs = {}
            if arg_str.strip():
                # Split by comma, handle both positional and keyword args
                for arg in re.split(r",(?![^\(\[]*[\]\)])", arg_str):
                    arg = arg.strip()
                    if "=" in arg:
                        k, v = arg.split("=", 1)
                        kwargs[k.strip()] = ast.literal_eval(v.strip())
                    else:
                        args.append(ast.literal_eval(arg))
            result = TaskExecutor.function_map[func_name](llm_interactor, *args, **kwargs)  # self as first argument
            results.append(str(result))
        return results

In [12]:
# TaskExecutor.execute_prompt(
#     fcm,
#     "Go make new deck name Geography and add a new card (flag: Turquoise) with question What is the capital of France? and answer Paris. The state should be 'New'."
# )

## Do the tests!

## First, we need to find out how to compare decks.

In [13]:

@dataclass(frozen=True)
class HashableCard:
    question: str
    answer: str
    flag: str
    state: str


import openai


def fuzzy_match(expected_card: Test_Card_Fuzzy, actual_card: HashableCard) -> bool:
    required = [
        expected_card.cardState.lower() == actual_card.state.lower(),
        expected_card.flag.lower() == actual_card.flag.lower(),
        "question" in expected_card.field__fuzzymatch or expected_card.question == actual_card.question,
        "answer" in expected_card.field__fuzzymatch or expected_card.answer != actual_card.answer
    ]
    if not all(required): return False

    prompt = f"""Please evaluate the following two flashcards, and tell me, if they have the same content. It is fine if the spelling, the grammar, the length and the wording differs, as long as the cards contain roughly the same information. If these cards are quite similar, please end your response with "true", else with "false" (without quotation marks). Only the last word of your respone will be evaluated.

Card 1:
Question: {expected_card.question}
Answer: {expected_card.answer}

Card 2:
Question: {actual_card.question}
Answer: {actual_card.answer}

/no_think
"""

    client = openai.OpenAI(
        api_key="lm-studio",
        base_url="http://localhost:1234/v1"
    )

    response = client.chat.completions.create(
        model="qwen2.5-14b-instruct",  # Replace with your LM Studio model name if needed
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0,
        max_tokens=10,
    )
    result = response.choices[0].message.content
    end = result.rstrip(" \n.")[-5:].lower()

    if end == "false":
        return False
    if end[1:] == "true":
        return True

    raise ValueError(f"Unexpected LM Studio response: {result!r}")


In [14]:
from typing import List, TypeVar, Callable, Any

LEFT = TypeVar("LEFT")
RIGHT = TypeVar("RIGHT")


# O(max(n, m))
# assumes left keys anr right keys are unique (means that necessarily left and right must be unique)
def match_by_key(
        left: list[LEFT],
        right: list[RIGHT],
        left_key: Callable[[LEFT], Any] = lambda x: x,
        right_key: Callable[[RIGHT], Any] = lambda x: x
) -> tuple[list[tuple[LEFT, RIGHT]], list[LEFT], list[RIGHT]]:
    leftByKey = {left_key(it): it for it in left}
    rightByKey = {right_key(it): it for it in right}

    if len(leftByKey) != len(left) or len(rightByKey) != len(right):
        raise ValueError("left and right must consist of unique elements that produce unique keys.")

    left_keys = set(leftByKey.keys())
    right_keys = set(rightByKey.keys())

    common_keys = left_keys & right_keys
    only_left_keys = left_keys - common_keys
    only_right_keys = right_keys - common_keys

    matches = [(leftByKey[key], rightByKey[key]) for key in common_keys]
    only_left = [leftByKey[key] for key in only_left_keys]
    only_right = [rightByKey[key] for key in only_right_keys]

    return matches, only_left, only_right


# O( n * m )
def match_by_equals(
        left: list[LEFT],
        right: list[RIGHT],
        equals: Callable[[LEFT, RIGHT], bool]
) -> tuple[list[tuple[LEFT, RIGHT]], list[LEFT], list[RIGHT]]:
    matches = []
    left_matched = len(left) * [False]
    right_matched = len(right) * [False]

    for l_idx, l in enumerate(left):
        for r_idx, r in enumerate(right):
            if equals(l, r):
                if left_matched[l_idx]:
                    raise ValueError(f"Left element #{l_idx}: {l} has multiple matches.")
                if right_matched[r_idx]:
                    raise ValueError(f"Right element #{r_idx}: {r} has multiple matches.")
                left_matched[l_idx] = True
                right_matched[r_idx] = True
                matches.append((l, r))
    only_left = [l for l_idx, l in enumerate(left) if not left_matched[l_idx]]
    only_right = [r for r_idx, r in enumerate(right) if not right_matched[r_idx]]

    return matches, only_left, only_right

In [15]:

print(match_by_equals([1, 3, 5], ["5", "7", "1", "9"], lambda l, r: l == int(r)))

print(match_by_key([1, 3, 5], ["5", "7", "1", "9"], right_key=lambda x: int(x)))


([(1, '1'), (5, '5')], [3], ['7', '9'])
([(1, '1'), (5, '5')], [3], ['9', '7'])


In [16]:
def compare_decks(expected: Test_Deck, actual: Deck) -> List[str]:
    actual_hash = [HashableCard(x.question, x.answer, x.flag, x.cardState) for x in actual.cards]
    exp_strict_hash = [HashableCard(x.question, x.answer, x.flag, x.cardState) for x in expected.cards if
                       not x.field__fuzzymatch]
    exp_fuzzy = [x for x in expected.cards if x.field__fuzzymatch]

    # match exact
    (_, unm_exp, tmp_unm_act) = match_by_key(exp_strict_hash, actual_hash)

    (_, unm_exp_fuzzy, final_unm_act) = match_by_equals(exp_fuzzy, tmp_unm_act, equals=fuzzy_match)

    # Now create the error messages
    errors = []
    for additional_fuzzy in unm_exp_fuzzy:
        errors += [f"The following expected, fuzzy-matching card has not found a partner:\nf{additional_fuzzy}"]

    for additional_expected in unm_exp:
        errors += [f"The following expected card has not found a partner:\n{additional_expected}"]

    for additional_actual in final_unm_act:
        errors += [f"The following provided card was not expected:\n{additional_actual}"]
    return errors


def get_test_deck_by_key(key: str, test_data: Test_Data):
    deck = test_data.test_decks[key]
    if deck is None:
        raise ValueError(f"Expected deck '{key}' does not exist.")

    return Test_Deck(
        name=deck.name,
        cards=[Test_Card_Fuzzy(question=it.question, answer=it.answer, flag=it.flag, cardState=it.cardState)
               for it in deck.cards]
    )


def evaluate_test_result(expected: Test_ExpectedResult, test_data: Test_Data, actual: FlashcardManager):
    errors = []

    expected_decks: list[Test_Deck] = []
    for expected_deck in expected.decks:
        expected_decks += [
            get_test_deck_by_key(expected_deck, test_data) if isinstance(expected_deck, str) else expected_deck]

    # now match expected decks to actual decks using name
    (matched, unmatched_expected, unmatched_actual) = match_by_key(
        expected_decks, actual.get_decks(),
        lambda l: l.name, lambda r: r.name
    )

    for (expected, actual) in matched:
        errors += compare_decks(expected, actual)

    # create unmatched error messages
    for unmatched_expected_deck in unmatched_expected:
        errors += [f"The deck {unmatched_expected_deck.name} was expected, but was not in the actual result."]

    for unmatched_actual_deck in unmatched_actual:
        errors += [f"The deck {unmatched_actual_deck} was in the actual result, but was unexpected."]

    return errors


In [17]:
def foo():  # else 'expected' and 'actual' are in the global scope and I get 5 million warnings. Even if del.
    expected = Test_Card_Fuzzy(
        question="Was ist ein Integer?",
        answer="Eine ganze Zahl.",
        flag="None",
        cardState="New"
    )

    actual = HashableCard(
        question="Was ist ein Integer?",
        answer="Eine Zahl ohne Nachkommastellen",
        flag="None",
        state="New"
    )

    print(fuzzy_match(expected, actual))


foo()

True


In [18]:
import itertools


# copied from recording.py
def replace_many(s: str, replacements: dict) -> str:
    for old, new in replacements.items():
        s = s.replace(old, new)
    return s


def result_contains_placeholders(expected_result: Test_ExpectedResult, replacement_keys: set[str]):
    regexp = re.compile(f"""<({"|".join(replacement_keys)})>""")
    repr = expected_result.model_dump_json(by_alias=True)
    return regexp.search(repr) is not None


def replace_many_in_result(expected_result: Test_ExpectedResult, replacements: dict) -> Test_ExpectedResult:
    repr = expected_result.model_dump_json(by_alias=True)
    replaced_repr = replace_many(repr, replacements)
    return Test_ExpectedResult.model_validate_json(replaced_repr)


def get_prompt_with_parameters(
        prompt: str, parameters: dict[str, list[str]], expected_result: Test_ExpectedResult
) -> list[tuple[str, Test_ExpectedResult]]:
    if len(parameters) == 0:
        return [(prompt, expected_result)]

    keys = parameters.keys()
    keysWithAngles = [f"<{it}>" for it in keys]
    values = parameters.values()

    only_zip = "join" not in parameters or parameters.pop("join") == "zip"

    if not only_zip:  # cross product
        combinations = itertools.product(*values)
    else:  # zip
        assert len({len(it) for it in parameters.values()}) == 1, (
            "all parameters must have the same length"
        )
        combinations = zip(*values)

    substitutions = [
        dict(zip(keysWithAngles, combination)) for combination in combinations
    ]

    if not result_contains_placeholders(expected_result, set(keys)):
        return [(replace_many(prompt, params), expected_result) for params in substitutions]

    # print("Found a result with parameters.")
    # print(expected_result)
    res = []
    for subs in substitutions:
        res += [(replace_many(prompt, subs), replace_many_in_result(expected_result, subs))]
    return res

In [19]:
from dataclasses import asdict
import json


def get_environment(envir_name: str, test_data: Test_Data, flashcard_manager: FlashcardManager):
    deck_names = [test_data.test_decks[key].name for key in test_data.dummy_environments[envir_name].decks]
    res = flashcard_manager.copy()

    for res_deck in res.get_decks():
        if res_deck.name not in deck_names:
            res.delete_deck(res_deck)

    return res


@dataclass(frozen=True)
class TestInfo:
    passed: bool
    crashed: bool
    query: str
    error_messages: list[str]


def execute_tests(test_data: Test_Data, first_n=None):
    tests_run = 0
    res: list[TestInfo] = []

    try:
        for test_nr, test in enumerate(test_data.tests):
            print(f"###### TEST NUMBER {test_nr} out of {len(test_data.tests)} ({tests_run} total runs incl. parameters so far)")
            # TODO: Only single-turn prompts for now
            if len(test.queries) > 1:
                continue

            for query in test.queries[0]:
                for (finished_query, finished_expected) in \
                        get_prompt_with_parameters(query, test.params or dict(), test.expected_result):
                    try:
                        test_flashcard_manager = get_environment(test.environment, test_data, fcm)
                        TaskExecutor.execute_prompt(test_flashcard_manager, finished_query)

                        errors = evaluate_test_result(finished_expected, test_data, test_flashcard_manager)
                        res += [TestInfo(len(errors) == 0, False, finished_query, errors)]
                    except Exception as e:
                        res += [TestInfo(False, True, finished_query, [str(e)])]

                    tests_run += 1
                    if first_n is not None and tests_run >= first_n:
                        return res
    except KeyboardInterrupt:
        return res

    return res


def execute_tests_write_json(test_data: Test_Data, out_file: str, first_n=None):
    res = execute_tests(test_data, first_n=first_n)
    with open(out_file, 'w', encoding='utf-8') as f:
        json.dump([asdict(item) for item in res], f, ensure_ascii=False, indent=4)
    return res

In [20]:
from datetime import datetime
from zoneinfo import ZoneInfo

now = datetime.now(ZoneInfo("Europe/Berlin")).strftime("%Y-%m-%d %H:%M:%S %z")
res = execute_tests_write_json(test_data, out_file=f"test report {now}.json")
res

###### TEST NUMBER 0 out of 23 (0 total runs incl. parameters so far)

=========== REQUEST ===========:
You are an assistant for a flashcard learning system. The system manages Cards, that are collected in Decks.

About Cards:
    A Card is a representation of a flashcard, containing a question and an answer. The card is uniquely identified by the id.

    Properties:
      id (str): The id uniquely identifies the card. It is represented as "card_xxxx_xxxx", with x being hexadecimal digits.
          The id is the only way to identify a card.
      question (str): The question (frontside) of the card.
      answer (str): The answer (frontside) of the card.
      flag (str): The flag of the card. **Must** be one of:
          none, red, orange, green, blue, pink, turquoise, purple
      cardState (str): The state of the card in the flashcard system. **Must** be one of:
          new, learning, review, suspended, buried
    

About Decks:
    A Deck represents a collection of flashcards.

[TestInfo(passed=True, crashed=False, query="Create a new deck with the name 'Natural Language Processing'.", error_messages=[])]

In [34]:
def foo():
    print(fcm)
    fcm2 = fcm.copy()
    print(get_environment("all", test_data, fcm))
    fcm2.add_deck("ONLY COPY")
    print(fcm2)
    print(fcm)
    print(get_environment("all", test_data, fcm))

    del fcm2


foo()

Flashcard Manager with the following decks:
* Deck 'Empty Deck' (id: deck_8f8c_8b72) containing 0 cards
* Deck 'Latin Literature' (id: deck_55a2_1335) containing 30 cards
* Deck 'Java Programming' (id: deck_a971_eb44) containing 103 cards
* Deck 'Java Programming 10' (id: deck_8486_e8b2) containing 10 cards
* Deck 'python' (id: deck_9653_ac70) containing 86 cards

Flashcard Manager with the following decks:
* Deck 'Latin Literature' (id: deck_0d55_e36b) containing 30 cards
* Deck 'Java Programming' (id: deck_c237_1899) containing 103 cards
* Deck 'python' (id: deck_1187_3fe7) containing 86 cards

Flashcard Manager with the following decks:
* Deck 'Empty Deck' (id: deck_c914_fcc4) containing 0 cards
* Deck 'Latin Literature' (id: deck_059d_671a) containing 30 cards
* Deck 'Java Programming' (id: deck_ca29_9dbe) containing 103 cards
* Deck 'Java Programming 10' (id: deck_200f_4a58) containing 10 cards
* Deck 'python' (id: deck_3f64_d8e8) containing 86 cards
* Deck 'ONLY COPY' (id: deck_a

In [ ]:
raise ValueError("Jo stop executing please")

## Audio

In [ ]:
import sounddevice as sd
import numpy as np
import scipy.io.wavfile as wav
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline


# Record audio from microphone
def record_audio(duration=5, fs=16000):
    print("Recording...")
    audio = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype='int16')
    sd.wait()
    print("Recording finished.")
    return audio.flatten(), fs


# Save audio to a temporary WAV file
def save_wav(audio, fs, filename="temp.wav"):
    wav.write(filename, fs, audio)
    return filename


def create_pipeline(only_cpu: bool = False) -> pipeline:
    device = "cuda:0" if not only_cpu and torch.cuda.is_available() else "cpu"
    torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
    model_id = "openai/whisper-medium"

    model = AutoModelForSpeechSeq2Seq.from_pretrained(
        model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
    )
    model.to(device)
    processor = AutoProcessor.from_pretrained(model_id)
    return pipeline(
        "automatic-speech-recognition",
        model=model,
        tokenizer=processor.tokenizer,
        feature_extractor=processor.feature_extractor,
        torch_dtype=torch_dtype,
        device=device,
    )


pipe = create_pipeline(only_cpu=False)


# Load and transcribe audio
def transcribe(filename):
    result = pipe(filename)
    print("Transcription:", result)

In [ ]:
audio, fs = record_audio(duration=10, fs=16000)
filename = save_wav(audio, fs)
transcribe(filename)

In [ ]:
# import sounddevice as sd
# import numpy as np
#
# def audio_stream(chunk_duration=1, fs=16000):
#     chunk_samples = int(chunk_duration * fs)
#     with sd.InputStream(samplerate=fs, channels=1, dtype='int16') as stream:
#         print("Live transcription started. Speak into the microphone.")
#         while True:
#             audio_chunk, _ = stream.read(chunk_samples)
#             yield audio_chunk.flatten()
#
# for chunk in audio_stream():
#     audio_float = (chunk / 32768.0).astype(np.float32)
#     # Pass as dict with 'array' and 'sampling_rate'
#     result = pipe({"array": audio_float, "sampling_rate": 16000})
#     print("You said:", result["text"])

In [23]:
type(data.tests[0])

NameError: name 'data' is not defined

{"decks":[{"name":"<name>","cards":[]}]}



{"decks":[{"name":"empty_deck","cards":[{"question":"<question>","answer":"<answer>","flag":"None","cardState":"New","field__fuzzymatch":[]}]}]}



{"decks":[{"name":"<name>","cards":[{"question":"<question>","answer":"<answer>","flag":"<flag>","cardState":"New","field__fuzzymatch":[]}]}]}



{"decks":[{"name":"Java Programming 10","cards":[{"question":"What is the purpose of the 'main' method in Java?","answer":"It's the entry point to start program execution.","flag":"None","cardState":"New","field__fuzzymatch":[]},{"question":"How do you declare a variable in Java?","answer":"Specify type, then variable name, e.g. int x;","flag":"None","cardState":"New","field__fuzzymatch":[]},{"question":"What is a class in Java?","answer":"A blueprint for creating objects.","flag":"None","cardState":"New","field__fuzzymatch":[]},{"question":"What is the difference between '==' and 'equals()' in Java?","answer":"'==' compares references; 'equals()' compar